Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [4]:
pickle_file = 'data/notMNIST.pkl'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [5]:
IMAGE_SIZE = 28
NUM_LABELS = 10
NUM_CHANNELS = 1 #greyscale


def reformat(dataset, labels):
  dataset = np.reshape(dataset, (-1, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)).astype(np.float32)
  labels = (np.arange(NUM_LABELS) == labels[:, None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [9]:
BATCH_SIZE = 16
PATCH_SIZE = 5
DEPTH = 16
NUM_HIDDEN = 64

In [12]:
def cnn_2_layers(tf_train_dataset, tf_train_labels, tf_valid_dataset, tf_test_dataset):
  # Variables
  ## First Convolution Layer
  layer1_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, PATCH_SIZE, NUM_CHANNELS, DEPTH], stddev=0.1))
  layer1_bias = tf.Variable(tf.zeros([DEPTH]))
  ## Second Convolution Layer
  layer2_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, PATCH_SIZE, DEPTH, DEPTH], stddev=0.1))
  layer2_bias = tf.Variable(tf.constant(1.0, shape=[DEPTH]))
  ## Fully Connected Layer
  layer3_weights = tf.Variable(tf.truncated_normal([IMAGE_SIZE // 4 * IMAGE_SIZE // 4 * DEPTH, NUM_HIDDEN], stddev=0.1))
  layer3_bias = tf.Variable(tf.constant(1.0, shape=[NUM_HIDDEN]))
  ## Output Layer
  layer4_weights = tf.Variable(tf.truncated_normal([NUM_HIDDEN, NUM_LABELS], stddev=0.1))
  layer4_bias = tf.Variable(tf.constant(1.0, shape=[NUM_LABELS]))

  # Model
  def model(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, strides=[1, 2, 2, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_bias)
    conv2 = tf.nn.conv2d(hidden1, layer2_weights, strides=[1, 2, 2, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_bias)
    fcc_shape = hidden2.get_shape().as_list()
    hidden2 = tf.reshape(hidden2, [fcc_shape[0], fcc_shape[1] * fcc_shape[2] * fcc_shape[3]])
    hidden3 = tf.nn.relu(tf.matmul(hidden2, layer3_weights) + layer3_bias)
    return tf.matmul(hidden3, layer4_weights) + layer4_bias

  # Training Computation
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  # Optimizer
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

  # Predictions
  train_predictions = tf.nn.softmax(logits)
  test_predictions = tf.nn.softmax(model(tf_test_dataset))
  valid_predictions = tf.nn.softmax(model(tf_valid_dataset))
  return optimizer, loss, train_predictions, valid_predictions, test_predictions


def conv_net(num_steps, cnn_type):
  graph = tf.Graph()
  with graph.as_default():
    # Input
    tf_train_dataset = tf.placeholder(tf.float32, shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS))
    tf_train_labels = tf.placeholder(tf.float32, shape=(BATCH_SIZE, NUM_LABELS))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    optimizer, loss, train_predictions, valid_predictions, \
    test_predictions = cnn_type(tf_train_dataset, tf_train_labels, tf_valid_dataset, tf_test_dataset)

  with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in xrange(num_steps):
      offset = (step * BATCH_SIZE) % (train_labels.shape[0] - BATCH_SIZE)
      batch_data = train_dataset[offset:(offset + BATCH_SIZE), :, :, :]
      batch_labels = train_labels[offset:(offset + BATCH_SIZE), :]
      feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
      _, l, predictions = session.run([optimizer, loss, train_predictions], feed_dict=feed_dict)
      if step % 50 == 0:
        print('Minibatch loss at step %d: %f' % (step, l))
        print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
        print('Validation accuracy: %.1f%%' % accuracy(valid_predictions.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_predictions.eval(), test_labels))


In [13]:
NUM_STEPS = 1001
conv_net(NUM_STEPS, cnn_2_layers)

Initialized
Minibatch loss at step 0: 3.128249
Minibatch accuracy: 6.2%
Validation accuracy: 10.3%
Minibatch loss at step 50: 1.503863
Minibatch accuracy: 50.0%
Validation accuracy: 48.0%
Minibatch loss at step 100: 0.988977
Minibatch accuracy: 56.2%
Validation accuracy: 68.6%
Minibatch loss at step 150: 0.352969
Minibatch accuracy: 93.8%
Validation accuracy: 75.0%
Minibatch loss at step 200: 1.010936
Minibatch accuracy: 68.8%
Validation accuracy: 78.5%
Minibatch loss at step 250: 1.186480
Minibatch accuracy: 62.5%
Validation accuracy: 79.1%
Minibatch loss at step 300: 0.347087
Minibatch accuracy: 87.5%
Validation accuracy: 79.0%
Minibatch loss at step 350: 0.505745
Minibatch accuracy: 93.8%
Validation accuracy: 77.3%
Minibatch loss at step 400: 0.258349
Minibatch accuracy: 87.5%
Validation accuracy: 80.0%
Minibatch loss at step 450: 0.926782
Minibatch accuracy: 75.0%
Validation accuracy: 79.8%
Minibatch loss at step 500: 0.885467
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [14]:
def cnn_2_layers_maxpool(tf_train_dataset, tf_train_labels, tf_valid_dataset, tf_test_dataset):
  # Variables
  ## First Convolution Layer
  layer1_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, PATCH_SIZE, NUM_CHANNELS, DEPTH], stddev=0.1))
  layer1_bias = tf.Variable(tf.zeros([DEPTH]))
  ## Second Convolution Layer
  layer2_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, PATCH_SIZE, DEPTH, DEPTH], stddev=0.1))
  layer2_bias = tf.Variable(tf.constant(1.0, shape=[DEPTH]))
  ## Fully Connected Layer
  layer3_weights = tf.Variable(tf.truncated_normal([IMAGE_SIZE // 4 * IMAGE_SIZE // 4 * DEPTH, NUM_HIDDEN], stddev=0.1))
  layer3_bias = tf.Variable(tf.constant(1.0, shape=[NUM_HIDDEN]))
  ## Output Layer
  layer4_weights = tf.Variable(tf.truncated_normal([NUM_HIDDEN, NUM_LABELS], stddev=0.1))
  layer4_bias = tf.Variable(tf.constant(1.0, shape=[NUM_LABELS]))

  # Model
  def model(data):
    conv1 = tf.nn.conv2d(data, layer1_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_bias)
    hidden1 = tf.nn.max_pool(hidden1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    conv2 = tf.nn.conv2d(hidden1, layer2_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_bias)
    hidden2 = tf.nn.max_pool(hidden2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    fcc_shape = hidden2.get_shape().as_list()
    hidden2 = tf.reshape(hidden2, [fcc_shape[0], fcc_shape[1] * fcc_shape[2] * fcc_shape[3]])
    hidden3 = tf.nn.relu(tf.matmul(hidden2, layer3_weights) + layer3_bias)
    return tf.matmul(hidden3, layer4_weights) + layer4_bias

  # Training Computation
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  # Optimizer
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

  # Predictions
  train_predictions = tf.nn.softmax(logits)
  test_predictions = tf.nn.softmax(model(tf_test_dataset))
  valid_predictions = tf.nn.softmax(model(tf_valid_dataset))
  return optimizer, loss, train_predictions, valid_predictions, test_predictions

In [15]:
conv_net(1001, cnn_2_layers_maxpool)

Initialized
Minibatch loss at step 0: 3.693719
Minibatch accuracy: 6.2%
Validation accuracy: 14.6%
Minibatch loss at step 50: 1.610906
Minibatch accuracy: 43.8%
Validation accuracy: 53.9%
Minibatch loss at step 100: 1.028045
Minibatch accuracy: 68.8%
Validation accuracy: 61.3%
Minibatch loss at step 150: 0.571527
Minibatch accuracy: 81.2%
Validation accuracy: 74.7%
Minibatch loss at step 200: 1.148425
Minibatch accuracy: 68.8%
Validation accuracy: 78.1%
Minibatch loss at step 250: 1.046703
Minibatch accuracy: 62.5%
Validation accuracy: 78.8%
Minibatch loss at step 300: 0.392758
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
Minibatch loss at step 350: 0.509496
Minibatch accuracy: 93.8%
Validation accuracy: 80.2%
Minibatch loss at step 400: 0.304147
Minibatch accuracy: 93.8%
Validation accuracy: 81.5%
Minibatch loss at step 450: 0.893579
Minibatch accuracy: 87.5%
Validation accuracy: 80.6%
Minibatch loss at step 500: 0.728942
Minibatch accuracy: 81.2%
Validation accuracy: 82.8%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [16]:
def le_net_5(tf_train_dataset, tf_train_labels, tf_valid_dataset, tf_test_dataset):
  keep_prob = 0.9
  beta = 0.01
  # Variables
  ## First Convolution Layer
  layer1_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, PATCH_SIZE, NUM_CHANNELS, DEPTH], stddev=0.1))
  layer1_bias = tf.Variable(tf.zeros([DEPTH]))
  ## Second Convolution Layer
  layer2_weights = tf.Variable(tf.truncated_normal([PATCH_SIZE, PATCH_SIZE, DEPTH, DEPTH], stddev=0.1))
  layer2_bias = tf.Variable(tf.constant(1.0, shape=[DEPTH]))
  ## Fully Connected Layer
  layer3_weights = tf.Variable(tf.truncated_normal([IMAGE_SIZE // 4 * IMAGE_SIZE // 4 * DEPTH, NUM_HIDDEN], stddev=0.1))
  layer3_bias = tf.Variable(tf.constant(1.0, shape=[NUM_HIDDEN]))
  ## Output Layer
  layer4_weights = tf.Variable(tf.truncated_normal([NUM_HIDDEN, NUM_LABELS], stddev=0.1))
  layer4_bias = tf.Variable(tf.constant(1.0, shape=[NUM_LABELS]))

  # Model
  def model(data, is_train=False):
    conv1 = tf.nn.conv2d(data, layer1_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden1 = tf.nn.relu(conv1 + layer1_bias)
    hidden1 = tf.nn.max_pool(hidden1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    if is_train:
      hidden1 = tf.nn.dropout(hidden1, keep_prob)
    conv2 = tf.nn.conv2d(hidden1, layer2_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden2 = tf.nn.relu(conv2 + layer2_bias)
    hidden2 = tf.nn.max_pool(hidden2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    if is_train:
      hidden2 = tf.nn.dropout(hidden2, keep_prob)
    fcc_shape = hidden2.get_shape().as_list()
    hidden2 = tf.reshape(hidden2, [fcc_shape[0], fcc_shape[1] * fcc_shape[2] * fcc_shape[3]])
    hidden3 = tf.nn.relu(tf.matmul(hidden2, layer3_weights) + layer3_bias)
    if is_train:
      hidden3 = tf.nn.dropout(hidden3, keep_prob)
    return tf.matmul(hidden3, layer4_weights) + layer4_bias

  # Training Computation
  logits = model(tf_train_dataset, is_train=True)
  l2_loss = tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_bias) \
            + tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer2_bias) \
            + tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_bias) \
            + tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer4_bias)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + beta * l2_loss)

  # Optimizer
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 1000, 0.90, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

  # Predictions
  train_predictions = tf.nn.softmax(logits)
  test_predictions = tf.nn.softmax(model(tf_test_dataset))
  valid_predictions = tf.nn.softmax(model(tf_valid_dataset))
  return optimizer, loss, train_predictions, valid_predictions, test_predictions

In [ ]:
conv_net(1001, le_net_5)

Initialized
Minibatch loss at step 0: 6.895959
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 4.831171
Minibatch accuracy: 12.5%
Validation accuracy: 12.8%
Minibatch loss at step 100: 4.403604
Minibatch accuracy: 43.8%
Validation accuracy: 46.2%
Minibatch loss at step 150: 3.502974
Minibatch accuracy: 50.0%
Validation accuracy: 57.8%
Minibatch loss at step 200: 3.361840
Minibatch accuracy: 62.5%
Validation accuracy: 70.0%
Minibatch loss at step 250: 3.525927
Minibatch accuracy: 62.5%
Validation accuracy: 75.1%
Minibatch loss at step 300: 2.584792
Minibatch accuracy: 81.2%
Validation accuracy: 79.3%
Minibatch loss at step 350: 2.662890
Minibatch accuracy: 87.5%
Validation accuracy: 78.2%
Minibatch loss at step 400: 2.197473
Minibatch accuracy: 100.0%
Validation accuracy: 81.4%
Minibatch loss at step 450: 2.617240
Minibatch accuracy: 81.2%
Validation accuracy: 78.5%
Minibatch loss at step 500: 2.469269
Minibatch accuracy: 81.2%
Validation accuracy: 81.3%
M